In [38]:
import json
import smart_open
import pandas as pd
import os

# RESULTS_DIR = "/Users/ianm/projects/ppl-suite-paper-runs/results"
# MODEL_PATH = "olmo/1b/olmo-small-dolma-150B-mcli-results"
RESULTS_DIR = "/Users/ianm/projects/ppl-suite-paper-runs/results"
MODEL_PATH = "pythia/1b"

model_path = os.path.join(RESULTS_DIR, MODEL_PATH)

for sheet_name in os.listdir(model_path):
    # if "c4" not in sheet_name:
    #     continue
    print(sheet_name)


ppl_token_ppl_dolma_100_programing_languages_subdomains.jsonl.gz
ppl_byte_ppl_twitterAAE_HELM_fixed_subdomains.jsonl.gz
ppl_ptb_token_count_avg_logits.jsonl.gz
ppl_redpajama_token_count_avg_logits.jsonl.gz
ppl_word_ppl_c4_100_domains_subdomains.jsonl.gz
ppl_dolma_100_subreddits_token_count_avg_logits.jsonl.gz
ppl_byte_ppl_redpajama_subdomains.jsonl.gz
ppl_char_ppl_m2d2_wikipedia_unsplit_subdomains.jsonl.gz
bits_per_byte_ppl_pile_subdomains.jsonl.gz
ppl_char_ppl_dolma_100_programing_languages_subdomains.jsonl.gz
ppl_byte_ppl_dolma_100_programing_languages_subdomains.jsonl.gz
bits_per_byte_ppl_dolma_100_programing_languages_subdomains.jsonl.gz
ppl_char_ppl_m2d2_s2orc_unsplit_subdomains.jsonl.gz
bits_per_byte_ppl_c4_100_domains_subdomains.jsonl.gz
ppl_char_ppl_ice_fixed_subdomains.jsonl.gz
ppl_dolma-v1_5_token_count_avg_logits.jsonl.gz
ppl_word_ppl_ice_fixed_subdomains.jsonl.gz
ppl_4chan_meta_sep_token_count_avg_logits.jsonl.gz
ppl_word_ppl_redpajama_subdomains.jsonl.gz
ppl_word_ppl_dolma

In [40]:
def standardize_model_metadata(row):
    if row['model'].startswith('s3://ai2-llm/checkpoints/1b/olmo-small-dolma-150B-mcli'):
        if 'limit' in row:
            assert row['limit'] is None
            del row['limit']
        if 'model_kwargs' in row:
            assert row['model_kwargs'] == {'revision': None, 'trust_remote_code': True}
            del row['model_kwargs']
        if 'full_model' in row:
            del row['full_model']
        if 'random_subsample_seed' in row:
            del row['random_subsample_seed']
        if 'tango_step' in row:
            del row['tango_step']
        if 'tango_workspace' in row:
            del row['tango_workspace']
        if 'model_max_length' in row:
            assert row['model_max_length'] == 2048
            del row['model_max_length']
        if 'max_batch_tokens' in row:
            del row['max_batch_tokens']
        if 'batch_size' in row:
            del row['batch_size']
        if 'processing_time' in row:
            del row['processing_time']
        original_model_name = row['model']
        # row['original_model_name'] = original_model_name
        row['model'] = 'olmo'
        row['model_size'] = 1177240704 # provided by Ananya non embedding params
        row['training_steps'] = int(original_model_name.split('/')[-1].split('-')[0].replace('step', ''))
        row['tokens_seen'] = row['training_steps'] * 4325376 # max_sequence_length: 2048 * global_train_batch_size: 2112 = 4,325,376
        row['training_dataset'] = 'Dolma V1.5'
    elif row['model'].startswith('EleutherAI/pythia-1b'):
        if 'limit' in row:
            assert row['limit'] is None
            del row['limit']
        if 'full_model' in row:
            del row['full_model']
        if 'random_subsample_seed' in row:
            del row['random_subsample_seed']
        if 'tango_step' in row:
            del row['tango_step']
        if 'tango_workspace' in row:
            del row['tango_workspace']
        if 'model_max_length' in row:
            assert row['model_max_length'] == 2048
            del row['model_max_length']
        if 'max_batch_tokens' in row:
            del row['max_batch_tokens']
        if 'batch_size' in row:
            del row['batch_size']
        if 'processing_time' in row:
            del row['processing_time']
        original_model_name = row['model']
        # row['original_model_name'] = original_model_name
        row['model'] = 'pythia'
        row['model_size'] = 805736448 # non-embedding params 
        row['training_steps'] = int(row['model_kwargs']['revision'].replace('step', ''))
        row['tokens_seen'] = row['training_steps'] * 2097152 # from paper
        row['training_dataset'] = 'The Pile'

        if 'model_kwargs' in row:
            # assert row['model_kwargs'] == {'revision': , 'trust_remote_code': True}
            del row['model_kwargs']
    else:
        # pass
        raise ValueError(f"Unknown model: {row['model']}")
    return row

def read_jsonl(path):
    with smart_open.open(path, 'rt') as fin:
        for line in fin:
            data = standardize_model_metadata(json.loads(line))
            yield data
# ppl_redpajama_token_count_avg_logits = pd.DataFrame(read_jsonl(os.path.join(model_path, "ppl_redpajama_token_count_avg_logits.jsonl.gz")))
# ppl_redpajama_token_count_avg_logits.head()
ppl_token_ppl_twitterAAE_HELM_fixed_subdomains = pd.DataFrame(read_jsonl(os.path.join(model_path, "ppl_token_ppl_twitterAAE_HELM_fixed_subdomains.jsonl.gz")))
ppl_token_ppl_twitterAAE_HELM_fixed_subdomains

,date,model,task,num_instances,white,AA,split,model_size,training_steps,tokens_seen,training_dataset
0,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,165.399420,425.702129,test,805736448,10000,20971520000,The Pile
1,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,156.228514,396.766521,test,805736448,20000,41943040000,The Pile
2,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,145.941787,378.512562,test,805736448,30000,62914560000,The Pile
3,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,142.771728,371.075536,test,805736448,40000,83886080000,The Pile
4,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,140.450006,367.011482,test,805736448,50000,104857600000,The Pile
5,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,135.527131,353.785365,test,805736448,60000,125829120000,The Pile
6,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,131.253898,340.750409,test,805736448,70000,146800640000,The Pile
7,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,129.213084,337.059629,test,805736448,80000,167772160000,The Pile
8,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,134.823930,356.671968,test,805736448,90000,188743680000,The Pile
9,2023-10-31 08:37:39 UTC,pythia,ppl_twitterAAE_HELM_fixed,50000,127.744254,334.596307,test,805736448,100000,209715200000,The Pile


In [41]:
from functools import reduce

shared_cols = {
 'model',
 'model_size',
 'split',
 'tokens_seen',
 'training_dataset',
 'training_steps'
 }

non_domain_cols = set([
    "date",
    "model",
    "model_kwargs",
    "full_model",
    "task",
    "processing_time",
    "num_instances",
    "tango_workspace",
    "tango_step",
    "model_max_length",
    "max_batch_tokens",
    "batch_size",
    "limit",
    "split",
    "random_subsample_seed",
    "model_size",
    "training_steps",
    "tokens_seen",
    "training_dataset"
])

# TODO HANDLE datasets without subdomains as subdomains?

def get_metric_for_all_subdomains(sheet_names, metrics_name):
    dfs = []
    domain_cols = set()
    for k in sheet_names:
        if not k.endswith('subdomains.jsonl.gz'):
            continue
        if k.startswith(metrics_name):
            df = pd.DataFrame(read_jsonl(os.path.join(model_path, k)))
            
            df = df[list(df.columns.difference(non_domain_cols - shared_cols))]
            dfs.append(df)
    
    # join all the dfs on the domain cols, by concatenating the domain cols
    merged_df = reduce(lambda left,right: pd.merge(left,right,on=list(shared_cols), how='outer'), dfs)
    return merged_df

ppl_token_all_domains = get_metric_for_all_subdomains(os.listdir(model_path), 'ppl_token')
ppl_token_all_domains.head()

,00_text,01_markdown,02_c,03_php,04_java,05_c++,06_python,07_javascript,08_html,09_c#,10_yaml,11_go,12_typescript,13_xml,14_css,15_jupyter-notebook,16_rust,17_unity3d-asset,18_gettext-catalog,19_ruby,20_vue,21_sql,22_swift,23_kotlin,24_scala,25_scss,26_tex,27_dart,28_kicad,29_shell,30_smali,31_lua,32_restructuredtext,33_perl,34_diff,35_ini,36_jsx,37_haskell,38_gnuplot,39_postscript,40_groff,41_turtle,42_fortran,43_makefile,44_mathematica,45_pascal,46_common-lisp,47_gas,48_vhdl,49_julia,50_edn,51_visual-basic,52_powershell,53_g-code,54_ocaml,55_java-server-pages,56_solidity,57_graphviz-dot,58_less,59_twig,60_asciidoc,61_groovy,62_llvm,63_hcl,64_html+erb,65_erlang,66_elixir,67_eagle,68_arduino,69_coffeescript,70_toml,71_cuda,72_nix,73_smalltalk,74_cmake,75_actionscript,76_glsl,77_systemverilog,78_haxe,79_f#,80_max,81_objective-c++,82_standard-ml,83_dockerfile,84_emacs-lisp,85_scheme,86_clojure,87_handlebars,88_smarty,89_logos,90_stata,91_yacc,92_nimrod,93_tcl,94_viml,95_asp,96_protocol-buffer,97_r,98_cython,99_mediawiki,model,model_size,split,tokens_seen,training_dataset,training_steps,Culture_and_the_arts,Culture_and_the_arts__Culture_and_Humanities,Culture_and_the_arts__Games_and_Toys,Culture_and_the_arts__Mass_media,Culture_and_the_arts__Performing_arts,Culture_and_the_arts__Sports_and_Recreation,Culture_and_the_arts__The_arts_and_Entertainment,Culture_and_the_arts__Visual_arts,General_referece,General_referece__Further_research_tools_and_topics,General_referece__Reference_works,Health_and_fitness,Health_and_fitness__Exercise,Health_and_fitness__Health_science,Health_and_fitness__Human_medicine,Health_and_fitness__Nutrition,Health_and_fitness__Public_health,Health_and_fitness__Self_care,History_and_events,History_and_events__By_continent,History_and_events__By_period,History_and_events__By_region,Human_activites,Human_activites__Human_activities,Human_activites__Impact_of_human_activity,Mathematics_and_logic,Mathematics_and_logic__Fields_of_mathematics,Mathematics_and_logic__Logic,Mathematics_and_logic__Mathematics,Natural_and_physical_sciences,Natural_and_physical_sciences__Biology,Natural_and_physical_sciences__Earth_sciences,Natural_and_physical_sciences__Nature,Natural_and_physical_sciences__Physical_sciences,Philosophy_and_thinking,Philosophy_and_thinking__Philosophy,Philosophy_and_thinking__Thinking,Religion_and_belief_systems,Religion_and_belief_systems__Allah,Religion_and_belief_systems__Belief_systems,Religion_and_belief_systems__Major_beliefs_of_the_world,Society_and_social_sciences,Society_and_social_sciences__Social_sciences,Society_and_social_sciences__Society,Technology_and_applied_sciences,Technology_and_applied_sciences__Agriculture,Technology_and_applied_sciences__Computing,Technology_and_applied_sciences__Engineering,Technology_and_applied_sciences__Transport,books_x,common-crawl,pes2o,reddit_uniform,stack_uniform,wiki,00_AskReddit,01_politics,02_AmItheAsshole,03_worldnews,04_relationships,05_relationship_advice,06_news,07_leagueoflegends,08_todayilearned,09_TwoXChromosomes,10_personalfinance,11_changemyview,12_unpopularopinion,13_movies,14_Games,15_nba,16_pics,17_gaming,18_soccer,19_nfl,20_explainlikeimfive,21_conspiracy,22_atheism,23_AskMen,24_videos,25_sex,26_raisedbynarcissists,27_NoStupidQuestions,28_DestinyTheGame,29_anime,30_DnD,31_ukpolitics,32_funny,33_europe,34_canada,35_Christianity,36_SquaredCircle,37_AskWomen,38_legaladvice,39_JUSTNOMIL,40_technology,41_IAmA,42_wow,43_Parenting,44_exmormon,45_AdviceAnimals,46_childfree,47_unitedkingdom,48_ffxiv,49_dndnext,50_ADHD,51_loseit,52_asoiaf,53_BabyBumps,54_Advice,55_australia,56_CFB,57_offmychest,58_PublicFreakout,59_TrueOffMyChest,60_science,61_magicTCG,62_asktransgender,63_DotA2,64_neoliberal,65_whowouldwin,66_depression,67_WTF,68_pathofexile,69_PoliticalDiscussion,70_Libertarian,71_PurplePillDebate,72_Fitness,73_books,74_dogs,75_pcmasterrace,76_teenagers,77_stopdrinking,78_Overwatch,79_television,80_buildapc,81_askscience,82_programming,83_Guildwars2

In [42]:
# get avg over all subdomains (not in shared_cols)

def get_avg_over_all_domains(df):
    domain_cols = list(set(ppl_token_all_domains.columns) - shared_cols)
    new_df = pd.concat([df[list(shared_cols)],df[domain_cols].mean(axis=1)], axis=1)
    return new_df

ppl_token_all_domains_avg = get_avg_over_all_domains(ppl_token_all_domains)
ppl_token_all_domains_avg

,training_dataset,training_steps,model,tokens_seen,model_size,split,0
0,The Pile,10000,pythia,20971520000,805736448,test,19.129570
1,The Pile,20000,pythia,41943040000,805736448,test,17.400475
2,The Pile,30000,pythia,62914560000,805736448,test,16.643438
3,The Pile,40000,pythia,83886080000,805736448,test,16.168290
4,The Pile,50000,pythia,104857600000,805736448,test,15.857576
5,The Pile,60000,pythia,125829120000,805736448,test,15.538965
6,The Pile,70000,pythia,146800640000,805736448,test,15.257166
7,The Pile,80000,pythia,167772160000,805736448,test,15.026311
8,The Pile,90000,pythia,188743680000,805736448,test,14.947737
9,The Pile,100000,pythia,209715200000,805736448,test,14.619481


In [36]:
# get avg over all subdomains (not in shared_cols)

def get_avg_over_all_domains(df):
    domain_cols = list(set(ppl_token_all_domains.columns) - shared_cols)
    new_df = pd.concat([df[list(shared_cols)],df[domain_cols].mean(axis=1)], axis=1)
    return new_df

ppl_token_all_domains_avg = get_avg_over_all_domains(ppl_token_all_domains)
ppl_token_all_domains_avg

,training_dataset,training_steps,model,tokens_seen,model_size,split,0
0,Dolma V1.5,5000,olmo,21626880000,1000000000,test,19.755437
1,Dolma V1.5,10000,olmo,43253760000,1000000000,test,18.313378
2,Dolma V1.5,15000,olmo,64880640000,1000000000,test,17.000006
3,Dolma V1.5,20000,olmo,86507520000,1000000000,test,15.207001
4,Dolma V1.5,25000,olmo,108134400000,1000000000,test,14.731191
5,Dolma V1.5,30000,olmo,129761280000,1000000000,test,14.506977
6,Dolma V1.5,35000,olmo,151388160000,1000000000,test,14.307419


## Per token

In [3]:
df = pd.DataFrame(results['ppl_redpajama_token_count_avg_logits'])
df

,model,model_kwargs,subdomain,token,count,avg_logits
0,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__arxiv,Fund,1,-16.910574
1,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__arxiv,OR,1,-16.874359
2,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__arxiv,:_,1,-15.613811
3,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__arxiv,Being,1,-15.429646
4,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__arxiv,initial,2,-15.279208
...,...,...,...,...,...,...
68336,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__wikipedia,arde,1,-0.000095
68337,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__wikipedia,assadors,1,-0.000056
68338,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__wikipedia,inth,1,-0.000053
68339,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__wikipedia,ioned,1,-0.000012


In [11]:
# show the top 10 tokens with the highest average logit
df.sort_values(by='avg_logits', ascending=False).head(10)

,model,model_kwargs,subdomain,token,count,avg_logits
17134,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__books,iece,1,-2.384186e-07
49036,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,BASIS,2,-1.728533e-06
57206,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__stackexchange,dtd,1,-2.503392e-06
49035,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,OR,2,-4.827956e-06
44559,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__commoncrawl,rest,1,-4.887569e-06
17133,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__books,oys,2,-6.258457e-06
49034,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,OF,2,-6.437280e-06
49033,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,ANY,2,-8.106198e-06
49032,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,KIND,2,-8.165800e-06
49031,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,licenses,2,-8.463824e-06


In [10]:
# show most common tokens
df.sort_values(by='count', ascending=False).head(10)

,model,model_kwargs,subdomain,token,count,avg_logits
48235,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,\n,7404,-0.205430
55650,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__stackexchange,\n,5984,-0.636155
48309,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,.,5761,-0.165911
6112,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__arxiv,\n,4750,-0.555916
15277,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__books,",",4710,-1.207286
48272,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__github,_,4546,-0.183955
28713,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__c4,.,4319,-0.959059
15235,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__books,the,4234,-1.254945
55771,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__stackexchange,.,3648,-0.534855
66822,EleutherAI/pythia-1b,"{'revision': 'step140000', 'trust_remote_code'...",redpajama__wikipedia,\n,3601,-0.719567


## MISC

In [ ]:
# import json
# import smart_open
# import pandas as pd


# # with gzip.open('toy_token_avg_logits_data.json.gz', 'rt') as fin: 
# with smart_open.open('/Users/ianm/projects/ppl-suite-paper-runs/olmo-small-dolma-150B-mcli-results.json.gz', 'rt') as fin:
# # with smart_open.open('/Users/ianm/projects/ppl-suite-paper-runs/test_output.json.gz', 'rt') as fin:
#     results = json.load(fin)

In [ ]:
# import os
# os.makedirs('olmo-small-dolma-150B-mcli-results', exist_ok=True)
# for table_name in results:
#     output_file = os.path.join('olmo-small-dolma-150B-mcli-results', table_name + ".jsonl.gz")
#     with smart_open.open(output_file, "wb") as f:
#         print(f"writing {table_name} to file")
#         for row in results[table_name]:
#             f.write(json.dumps(row).encode())
#             f.write(b"\n")